<a href="https://colab.research.google.com/github/frey50/Localizer/blob/main/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ──────────────────────────────────────────────────────────────
#  🔁 NLLB 1.8B + OpenChat Mistral 7B Pipeline
#      Uzbek (uzn_Latn) ⇄ English (eng_Latn)
#      Full system: Translation + Chatbot + Back Translation
#      Quantized models, no Fast tokenizer, ready to run.
# ──────────────────────────────────────────────────────────────

# ───── INSTALL DEPENDENCIES (if needed) ─────
# !pip install -U transformers sentencepiece bitsandbytes accelerate

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    BitsAndBytesConfig,
    NllbTokenizer
)

In [ ]:
# ───── 1. DEVICE + CONFIG ─────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ───── 2. LOAD TRANSLATOR (NLLB 1.8B, 4-bit) ─────
trans_model_id = "facebook/nllb-200-1.3B"  # Note: no official 1.8B, this is closest size
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
trans_tokenizer = NllbTokenizer.from_pretrained(trans_model_id)  # slow tokenizer
trans_model = AutoModelForSeq2SeqLM.from_pretrained(
    trans_model_id,
    device_map="auto",
    quantization_config=bnb_cfg
)

In [ ]:
# ───── 3. LOAD CHATBOT (OpenChat Mistral 7B) ─────
chat_model_id = "openchat/openchat-3.5-1210"
chat_tokenizer = AutoTokenizer.from_pretrained(chat_model_id, trust_remote_code=True)
chat_model = AutoModelForCausalLM.from_pretrained(
    chat_model_id,
    device_map="auto",
    quantization_config=bnb_config
)


In [ ]:
# ───── 4. LANG CODES ─────
UZ = "uzn_Latn"
EN = "eng_Latn"

# ───── 5. TRANSLATION FUNCTION ─────
def translate(text, src_lang, tgt_lang, prompt=""):
    trans_tokenizer.src_lang = src_lang
    input_text = (prompt + " " + text).strip()
    inputs = trans_tokenizer(input_text, return_tensors="pt").to(device)
    bos_id = trans_tokenizer.convert_tokens_to_ids(tgt_lang)
    outputs = trans_model.generate(
        **inputs,
        forced_bos_token_id=bos_id,
        max_length=256,
        num_beams=4
    )
    return trans_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()


In [ ]:
# ───── 6. OPENCHAT RESPONSE ─────
def chat_with_openchat(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
    tokens = chat_tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
    out_ids = chat_model.generate(tokens, max_new_tokens=256, do_sample=True)
    out_text = chat_tokenizer.decode(out_ids[0], skip_special_tokens=True)
    return out_text.split("<|assistant|>")[-1].strip()


In [ ]:
# ───── 7. FULL PIPELINE ─────
def uzbek_chatbot_pipeline(user_uz_text):
    # print(f"🧠 Input (Uzbek): {user_uz_text}")
    en_input = translate(user_uz_text, UZ, EN)
    # print(f"➡️ Translated to English: {en_input}")

    en_output = chat_with_openchat(en_input)
    # print(f"💬 LLM Output (English): {en_output}")

    uz_output = translate(en_output, EN, UZ)
    print(f"🔁 Back to Uzbek: {uz_output}")
    return uz_output


In [ ]:

# ───── 8. TEST LOOP ─────
if __name__ == "__main__":
    print("🌐 Uzbek-English Chatbot | Type 'q' to quit\n")
    while True:
        text = input("👤 > ")
        if text.lower().strip() == 'q':
            break
        uzbek_chatbot_pipeline(text)
        print("\n")